In [1]:
import json, sys
import numpy as np
import pandas as pd
from joblib import load


In [ ]:
# Config 
# Choose what to evaluate
TARGET = "wpa"        # or "epa"
SEASON = 2024         # or None for all seasons
POLICY = "eps"        # "greedy" or "eps"
EPS = 0.05            # epsilon for eps-greedy
CLIP = None           # e.g., 50 for weight clipping sensitivity, else None
BOOTSTRAP = 1000      # resamples for 95% CI

DATA_PATH = "data/decisions_2016_2024.csv"

# For sanity in prints
np.set_printoptions(suppress=True, linewidth=120)
pd.set_option("display.width", 120)

In [3]:
#Load artifacts (preprocessor, behavior policy, arm models, metadata)
pre = load("artifacts/preprocessor.joblib")
behavior = load("artifacts/behavior_policy.joblib")          # must expose predict_proba
arm_models = load(f"artifacts/arm_models_{TARGET}.joblib")   # dict: {'go': model, 'fg': model, 'punt': model}

with open("artifacts/metadata.json","r") as f:
    meta = json.load(f)

actions = meta["actions"]  # expected: ["go","punt","fg"] (order matters)
if set(actions) != {"go","punt","fg"}:
    raise ValueError(f"Unexpected actions in metadata: {actions}")

# exactly the columns used at train time
feature_cols = meta.get("feature_cols") or (meta["numeric_features"] + meta["categorical_features"])
print("Loaded artifacts.\nActions order:", actions, "\n#Features:", len(feature_cols))


Loaded artifacts.
Actions order: ['fg', 'go', 'punt'] 
#Features: 28


In [11]:
#Load evaluation data and select columns
df = pd.read_csv(DATA_PATH)
if SEASON is not None and "season" in df.columns:
    df = df.loc[df["season"].astype(int) == int(SEASON)].copy()

# Normalize action labels to the metadata actions
a_logged_raw = (
    df["action"]
      .astype(str).str.lower()
      .replace({"field_goal":"fg","fieldgoal":"fg","field goal":"fg",
                "go_for_it":"go","go-for-it":"go","go for it":"go"})
)
if not set(a_logged_raw.unique()).issubset(set(actions)):
    print("WARNING: Found actions not in metadata:", sorted(set(a_logged_raw.unique()) - set(actions)))

r = df[TARGET].astype(float).to_numpy()
X_raw = df[feature_cols].copy()

print("Eval rows:", len(df), "| Reward column:", TARGET)
print(df.columns.tolist())
df.head(2)


Eval rows: 3634 | Reward column: wpa
['season', 'week', 'game_id', 'game_date', 'play_id', 'posteam', 'defteam', 'home_team', 'away_team', 'posteam_type', 'qtr', 'game_seconds_remaining', 'half_seconds_remaining', 'quarter_seconds_remaining', 'down', 'ydstogo', 'yardline_100', 'score_differential', 'home_timeouts_remaining', 'away_timeouts_remaining', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'roof', 'surface', 'temp', 'wind', 'play_type', 'punt_attempt', 'field_goal_attempt', 'rush_attempt', 'pass_attempt', 'epa', 'wpa', 'success', 'yards_gained', 'first_down', 'touchdown', 'field_goal_result', 'kick_distance', 'punt_inside_twenty', 'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch', 'return_yards', 'penalty', 'aborted_play', 'play_deleted', 'goal_to_go', 'timeout', 'timeout_team', 'off_epa_4w', 'def_epa_4w', 'plays_in_drive_so_far', 'play_elapsed_s', 'game_time_elapsed', 'def_time_on_field_cum', 'def_time_on_field_share', 'is_q4_or_later', 'action', 'fg_pct_shor

,season,week,game_id,game_date,play_id,posteam,defteam,home_team,away_team,posteam_type,...,play_elapsed_s,game_time_elapsed,def_time_on_field_cum,def_time_on_field_share,is_q4_or_later,action,fg_pct_short,fg_pct_mid,fg_pct_long,punt_net_4w
28215,2024,1,2024_01_ARI_BUF,2024-09-08,823.0,ARI,BUF,BUF,ARI,away,...,0.0,900.0,757.0,0.841111,0,fg,0.924842,0.720459,0.719610,32.886762
28216,2024,1,2024_01_ARI_BUF,2024-09-08,1120.0,BUF,ARI,BUF,ARI,home,...,40.0,1209.0,440.0,0.363937,0,fg,0.950380,0.738880,0.652809,30.212951


In [5]:
#Build design matrix (using the same fitted preprocessor)
X_design = pre.transform(X_raw)  # sparse or dense depending on your preprocessor
X_design.shape

(3634, 160)

In [6]:
#Behavior policy probabilities π_b(a|x)

# Works whether `behavior` is a Pipeline (with pre inside) or a bare estimator (expects preprocessed X)
try:
    P_beh_raw = behavior.predict_proba(X_raw)        # if Pipeline
except Exception:
    P_beh_raw = behavior.predict_proba(X_design)     # if bare estimator

# Get classes from the behavior model
if hasattr(behavior, "classes_"):
    beh_classes = list(behavior.classes_)
else:
    beh_lr = getattr(getattr(behavior, "named_steps", {}), "get", lambda _: None)("logisticregression")
    beh_classes = list(getattr(beh_lr, "classes_", []))
    if not beh_classes:
        raise RuntimeError("Could not infer classes_ from behavior model.")

# Reindex to metadata actions order
colmap = {c:i for i,c in enumerate(beh_classes)}
P_beh = np.column_stack([P_beh_raw[:, colmap[a]] for a in actions])  # shape (N, K)
P_beh[:3]



array([[0.93372766, 0.06308965, 0.00318269],
       [0.97730246, 0.02092129, 0.00177625],
       [0.00014639, 0.00134649, 0.99850712]])

In [7]:
#Per-action value predictions μ̂(x,a) for all actions
def mu_matrix(arm_models, X_design, actions):
    N = X_design.shape[0]; K = len(actions)
    MU = np.zeros((N, K), dtype=float)
    for j, a in enumerate(actions):
        m = arm_models[a]
        if isinstance(m, tuple) and m[0] == "const":      # your code uses ("const", mu) when data is sparse
            MU[:, j] = float(m[1])
        else:
            MU[:, j] = m.predict(X_design)
    return MU

MU = mu_matrix(arm_models, X_design, actions)  # (N,3)
MU[:3]


array([[-0.01894225,  0.01925375,  0.01047604],
       [ 0.01155332,  0.00034312,  0.00285456],
       [ 0.02319527,  0.01586833,  0.01157821]])

In [8]:
#Candidate policy π_new(a|x): greedy or ε-greedy
def policy_greedy(MU):
    N, K = MU.shape
    idx = MU.argmax(axis=1)
    P = np.zeros_like(MU)
    P[np.arange(N), idx] = 1.0
    return P

def policy_eps_greedy(MU, eps=0.05):
    N, K = MU.shape
    P = np.full((N, K), eps/float(K))
    best = MU.argmax(axis=1)
    P[np.arange(N), best] += 1.0 - eps
    return P

P_eval = policy_greedy(MU) if POLICY == "greedy" else policy_eps_greedy(MU, eps=EPS)
P_eval[:3]


array([[0.01666667, 0.96666667, 0.01666667],
       [0.96666667, 0.01666667, 0.01666667],
       [0.96666667, 0.01666667, 0.01666667]])

In [9]:
#Tie logged actions to indices and compute estimators (DM, IPS, SNIPS, DR)

# map logged action strings to column indices
act_map = {a:i for i,a in enumerate(actions)}
a_idx = a_logged_raw.map(act_map).to_numpy()

def estimators(P_eval, P_beh, MU, r, a_idx, clip=None):
    eps = 1e-6
    N = len(r)
    pi_e_logged = P_eval[np.arange(N), a_idx]
    pi_b_logged = P_beh[np.arange(N), a_idx]
    w = pi_e_logged / np.clip(pi_b_logged, eps, None)

    if clip is not None:
        w_eff = np.clip(w, 0, float(clip))
    else:
        w_eff = w

    mu_logged = MU[np.arange(N), a_idx]
    mu_target = np.sum(P_eval * MU, axis=1)

    DM = mu_target.mean()
    IPS = np.mean(w_eff * r)
    SNIPS = (w_eff * r).sum() / (w_eff.sum() + eps)
    DR = np.mean(w_eff * (r - mu_logged) + mu_target)

    # Effective sample size on rows where eval policy puts positive mass
    mask_pos = pi_e_logged > 0
    w_pos = w[mask_pos]
    ESS = (w_pos.sum()**2) / (np.sum(w_pos**2) + eps) if w_pos.size else 0.0

    return DM, IPS, SNIPS, DR, ESS, w

DM, IPS, SNIPS, DR, ESS, w = estimators(P_eval, P_beh, MU, r, a_idx, clip=CLIP)
print(f"DM={DM:.6f}  IPS={IPS:.6f}  SNIPS={SNIPS:.6f}  DR={DR:.6f}  ESS={ESS:.1f}/{len(r)}")


DM=0.019459  IPS=-0.003502  SNIPS=-0.002789  DR=0.006058  ESS=24.7/3634


In [ ]:
#Bootstrap 95% CIs (recommended for the report)

def bootstrap_all(P_eval, P_beh, MU, r, a_idx, B=1000, alpha=0.05, seed=123, clip=None):
    rs = np.random.RandomState(seed)
    n = len(r)
    dm_l, ips_l, snips_l, dr_l = [], [], [], []
    for _ in range(B):
        idx = rs.randint(0, n, n)
        DM_b, IPS_b, SNIPS_b, DR_b, _, _ = estimators(P_eval[idx], P_beh[idx], MU[idx], r[idx], a_idx[idx], clip=clip)
        dm_l.append(DM_b); ips_l.append(IPS_b); snips_l.append(SNIPS_b); dr_l.append(DR_b)
    def ci(arr):
        return float(np.quantile(arr, alpha/2)), float(np.quantile(arr, 1 - alpha/2))
    return ci(dm_l), ci(ips_l), ci(snips_l), ci(dr_l)

ci_dm, ci_ips, ci_snips, ci_dr = bootstrap_all(P_eval, P_beh, MU, r, a_idx, B=BOOTSTRAP, clip=CLIP)
print(f"DM 95% CI    [{ci_dm[0]:.6f}, {ci_dm[1]:.6f}]")
print(f"IPS 95% CI   [{ci_ips[0]:.6f}, {ci_ips[1]:.6f}]")
print(f"SNIPS 95% CI [{ci_snips[0]:.6f}, {ci_snips[1]:.6f}]")
print(f"DR 95% CI    [{ci_dr[0]:.6f}, {ci_dr[1]:.6f}]")


DM 95% CI    [0.019037, 0.019943]
IPS 95% CI   [-0.033645, 0.018612]
SNIPS 95% CI [-0.021899, 0.017847]
DR 95% CI    [-0.026982, 0.029934]


In [12]:
#Compare against the baseline “behavior” policy
#  Evaluate baseline by setting P_eval = P_beh
P_eval_base = P_beh.copy()
DM_b, IPS_b, SNIPS_b, DR_b, ESS_b, _ = estimators(P_eval_base, P_beh, MU, r, a_idx)
print("Baseline (behavior) — DR:", DR_b)
print("Your policy          — DR:", DR)
print("Lift (WPA):", DR - DR_b)


Baseline (behavior) — DR: 0.0016606480107016685
Your policy          — DR: 0.0060584471560989565
Lift (WPA): 0.004397799145397288


In [13]:
#Context slices (to show where it helps)
bins = pd.cut(df['yardline_100'], [0,20,40,60,80,100], right=False)
for b in bins.cat.categories:
    mask = (bins == b).to_numpy()
    dm, ips, snips, dr, ess, _ = estimators(P_eval[mask], P_beh[mask], MU[mask], r[mask], a_idx[mask])
    print(f"{b}: DR={dr:.4f}, ESS={ess:.1f}/{mask.sum()}")


[0, 20): DR=0.0109, ESS=97.0/612
[20, 40): DR=0.0148, ESS=206.8/787
[40, 60): DR=0.0182, ESS=97.2/855
[60, 80): DR=0.0365, ESS=12.0/1093
[80, 100): DR=-0.1802, ESS=2.5/287
